In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# colorblind friendly colorscheme https://davidmathlogic.com/colorblind/#%23648FFF-%23785EF0-%23DC267F-%23FE6100-%23FFB000
colorscheme = ['#648FFF', '#785EF0', '#DC267F', '#FE6100', '#FFB000'] # px.colors.sequential.Viridis[::3]
symbols = ['circle', 'square', 'x', 'diamond', 'triangle-up']

df = pd.read_csv('./build/results.csv')
df.head()

|       	| READ FRAME 	| DEQUE 	| GREY 	| DETECT MOTION 	|
|:-----:	|:----------:	|:-----:	|:----:	|:-------------:	|
| 1080P 	|    9285    	|   1   	| 3758 	|      864      	|
|  720  	|    6010    	|   1   	| 1564 	|      308      	|
            
<br>

|       	| OPENCV_BLUR 	| BOX_BLUR 	|   H1   	| H2/3/4 	|   H1_7  	|
|:-----:	|:-----------:	|:--------:	|:------:	|:------:	|:-------:	|
| 1080P 	|     3846    	|   15128  	| 470467 	| 822913 	| 1520998 	|
|  720  	|     1888    	|   6281   	| 195519 	| 335527 	|  640675 	|

In [ ]:
estimated_1080 = 9285 + 1 + 3758 + 864
estimated_720 = 6010 + 1 + 1564 + 308

# Input
blur_time = 335527
estimated_service_time = estimated_720 + blur_time

# df = df[df['trial name'] == 'door_short-grey-h1-3iter-stream'] 9.95 speedup, 21.820 service time
# df = df[df['trial name'] == 'door_short-grey-h1-3iter-data']     10.72 speedup, 18 service time
# df = df[df['trial name'] == 'door_short-grey-h2-3iter-stream']  9.89 speedup, 35
# df = df[df['trial name'] == 'door_short-grey-h3-3iter-stream'] 10.22, 34
# df = df[df['trial name'] == 'door_short-grey-h4-3iter-stream'] 9.83, 35
# df = df[df['trial name'] == 'door_short-grey-h1_7-3iter-stream'] 11.06, 55
# df = df[df['trial name'] == 'door_short-grey-opencv_blur-3iter-stream'] 1.31, 6
# df = df[df['trial name'] == 'door_short-grey-box_blur-3iter-stream'] 2.07, 6500

# df = df[df['trial name'] == 'house720-grey-h1-3iter-stream']   # 15 speedup, 13.50 service time
# df = df[df['trial name'] == 'house720-grey-h1-3iter-data']   # 16.43 speedup, 12.19 service time
# df = df[df['trial name'] == 'house720-grey-h1_7-3iter-stream']   # 16.56 speedup, 36.85 service time
# df = df[df['trial name'] == 'house720-grey-h3-3iter-stream']   # 15.68 speedup, 22.58 service time
# df = df[df['trial name'] == 'house720-grey-h4-3iter-stream'] 
# df = df[df['trial name'] == 'house720-grey-opencv_blur-3iter-stream']   # 1.39 speedup, 5 service time
# df = df[df['trial name'] == 'house720-grey-opencv_blur-3iter-stream']   # 2.2 speedup, 5.2 service time

# df = df[df['trial name'] == 'door1080-grey-h1-3iter-stream']   # 21.88 speedup, 21.96k service time
df = df[df['trial name'] == 'house720-greyh1r-1iter-stream-omp']

In [ ]:
mean = df.groupby(["type", "threads"]).mean().reset_index().sort_values(by=["type", "threads"])
std = df.groupby(["type", "threads"]).std().reset_index().sort_values(by=["type", "threads"])

means = {name:d for name, d in mean.groupby(["type"])}
std = {name:d for name, d in std.groupby(["type"])}
types = means.keys()
threads = pd.unique(mean["threads"]).tolist()

def get_fig(y, title, xtitle, ytitle, log_x=True, log_y=True):
    types = means.keys()
    threads = pd.unique(mean["threads"]).tolist()
    items = means["seq"]["items"]

    fig = go.Figure()

    for type in types:
        if type == "seq":
            continue
        fig.add_trace(go.Scatter(x=threads, y=means[type][y], name=type, error_y=dict(type="data", array=std[type][y])))

     # Add ideal metric
    if y == "completion time":
        seq_completion_time = means["seq"]["completion time"].iloc[0]
        ideal_completion_time = [seq_completion_time / thread for thread in threads]
        fig.add_trace(go.Scatter(x=threads, y=ideal_completion_time, name="ideal", line=dict(dash="dash")))

        estimated = [estimated_service_time * items / thread for thread in threads]
        fig.add_trace(go.Scatter(x=threads, y=estimated, name="lower bound", line=dict(dash="dash")))
    elif y == "service time":
        seq_service_time = means["seq"]["service time"].iloc[0]
        ideal_service_time = [seq_service_time / thread for thread in threads]
        fig.add_trace(go.Scatter(x=threads, y=ideal_service_time, name="ideal", line=dict(dash="dash")))

        estimated = [estimated_service_time / thread for thread in threads]
        fig.add_trace(go.Scatter(x=threads, y=estimated, name="lower bound", line=dict(dash="dash")))
    elif y == "efficiency":
        ones = pd.Series(np.ones(len(threads)))
        fig.add_trace(go.Scatter(x=threads, y=ones, name="ideal", line=dict(dash="dash")))
    else:
        fig.add_trace(go.Scatter(x=threads, y=threads, name="ideal", line=dict(dash="dash")))

    for i, trace in enumerate(fig.data):
        # use different symbol and color for each trace
        trace.marker.symbol = symbols[i]
        trace.marker.size = 10
        trace.line.color = colorscheme[i]
        # trace.line.color = "black"

    fig.update_layout(
        title=title,
        xaxis_title=xtitle,
        yaxis_title=ytitle,
        xaxis = dict(
            tickmode = 'array',
            tickvals = threads,
        ),
    )

    if log_x:
        fig.update_xaxes(type="log")

    if log_y:
        fig.update_yaxes(type="log")

    return fig

In [ ]:
get_fig(y="speedup", title="Speedup vs. Number of threads", xtitle="# Threads", ytitle="Speedup")

In [ ]:
get_fig(y="service time", title="Service time vs Number of threads", xtitle="# Threads", ytitle="Time (µs)")

In [ ]:
get_fig(y="completion time", title="Completion time vs Number of threads", xtitle="# Threads", ytitle="Time (µs)")

In [ ]:
get_fig(y="scalability", title="Scalability vs. Number of threads", xtitle="# Threads", ytitle="Scalability")

In [ ]:
get_fig(y="efficiency", title="Efficiency vs. Number of threads", xtitle="# Threads", ytitle="Efficiency")